### 載入MNIST手寫數字數據集

In [17]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [18]:
import tensorflow as tf

### 定義模型函式

In [19]:
def weight_variable(shape, name):
    return tf.Variable(tf.truncated_normal(shape = shape, stddev = 0.1), name = name)

def bias_variable(shape, name):
    return tf.Variable(tf.constant(0.1, shape = shape),name = name)

### 參數定義

In [20]:
BATCH_SIZE = 256
G_DIM = 128
X_D = tf.placeholder(tf.float32,[None, 784])
X_G = tf.placeholder(tf.float32,[None, 128])

weights = {
    "w_d1":weight_variable([784,128],'w_d1'),
    "w_d2":weight_variable([128,1],'w_d2'),
    'w_g1':weight_variable([128,256],'w_g1'),
    'w_g2':weight_variable([256,784],'w_g2')
}

biases = {
    'b_d1':bias_variable([128],'b_d1'),
    'b_d2':bias_variable([1],'b_d2'),
    'b_g1':bias_variable([256],'b_g1'),
    'b_g2':bias_variable([784],'b_g2')
}

var_d = [weights['w_d1'], weights['w_d2'], biases['b_d1'], biases['b_d2']]
var_g = [weights['w_g1'], weights['w_g2'], biases['b_g1'], biases['b_g2']]

### 定義生成與對抗網路函式

In [21]:
def generator(z):
    h_g1 = tf.nn.relu(tf.add(tf.matmul(z, weights['w_g1']), biases['b_g1']))
    h_g2 = tf.nn.sigmoid(tf.add(tf.matmul(h_g1, weights['w_g2']), biases['b_g2']))
    
    return h_g2

def discriminator(x):
    h_d1 = tf.nn.relu(tf.add(tf.matmul(x, weights['w_d1']), biases['b_d1']))
    h_d2 = tf.nn.sigmoid(tf.add(tf.matmul(h_d1, weights['w_d2']), biases['b_d2']))
    
    return h_d2

### 損失函數與優化器

In [22]:
def sample_z(m, n):
    return np.random.uniform(-1.,1., size = [m,n])

g_sample = generator(X_G)
d_real = discriminator(X_D)
d_predict = discriminator(g_sample)

d_loss = -tf.reduce_mean(tf.log(d_real) + tf.log(1. - d_predict))
g_loss = -tf.reduce_mean(tf.log(d_predict))



d_opt = tf.train.AdamOptimizer(0.0005).minimize(d_loss, var_list = var_d)
g_opt = tf.train.AdamOptimizer(0.0001).minimize(g_loss, var_list = var_g)

### 實例化執行

In [24]:
import numpy as np
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(15000):
        batchx = mnist.train.next_batch(BATCH_SIZE)[0]
        
        _,d_loss_train = sess.run([d_opt, d_loss], feed_dict = {X_D : batchx, X_G : sample_z(BATCH_SIZE, G_DIM)})
        
        _,g_loss_train = sess.run([g_opt, g_loss], feed_dict = {X_G : sample_z(BATCH_SIZE, G_DIM)})
        
        if step%1000 == 0:
            print("Step:{},d_loss:{},g_loss:{}".format(step,d_loss_train,g_loss_train))

Step:0,d_loss:1.293395757675171,g_loss:2.4811692237854004
Step:1000,d_loss:0.012302594259381294,g_loss:5.603867530822754
Step:2000,d_loss:0.02856394462287426,g_loss:5.653247833251953
Step:3000,d_loss:0.015586093999445438,g_loss:7.324066162109375
Step:4000,d_loss:0.02663254365324974,g_loss:5.903477668762207
Step:5000,d_loss:0.02850026823580265,g_loss:6.719682216644287
Step:6000,d_loss:0.037759073078632355,g_loss:7.297340393066406
Step:7000,d_loss:0.03580639883875847,g_loss:7.952790260314941
Step:8000,d_loss:0.08058644086122513,g_loss:6.650460243225098
Step:9000,d_loss:0.0626363754272461,g_loss:6.963033676147461
Step:10000,d_loss:0.1053246259689331,g_loss:5.9197678565979
Step:11000,d_loss:0.14665135741233826,g_loss:5.885573387145996
Step:12000,d_loss:0.14797133207321167,g_loss:5.649303436279297
Step:13000,d_loss:0.18788129091262817,g_loss:5.428991317749023
Step:14000,d_loss:0.20852303504943848,g_loss:5.641643524169922
